# Amplitude embedding (simulator)

Main steps:

* initialize an image of the sky (frequency domain)
* apply 2D FT --> visibilities (Fourier domain)
* encode visibilities data into qubits using amplitude embedding (non-conventional domain)
* measure qubits on simulator --> expected outcomes (back to conventional domain)
* apply 2D IFT --> fidelity computation

In [1]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane import numpy as pnp

## 1) Generating an artificial image of the sky (frequency domain)

In [2]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask

In [3]:
#image of the sky filled with double precision complex floats ('complex64')
#pixels are set to low complex random values (image background/noise) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[4.4682701e+01+6.4761119e+00j 4.4682701e+01+6.4761119e+00j
  7.0026629e-03+4.7672801e-03j 4.4682701e+01+6.4761119e+00j]
 [4.4682701e+01+6.4761119e+00j 1.1725369e-03+2.6129559e-03j
  6.9397106e-03+6.7491229e-03j 6.6754194e-03+5.7727741e-03j]
 [8.1521086e-03+5.8284053e-04j 1.4647950e-03+3.8324092e-03j
  3.4888552e-03+6.1570187e-03j 4.5068348e-03+4.3197051e-03j]
 [4.4682701e+01+6.4761119e+00j 4.0000589e-03+2.3147690e-03j
  3.1959236e-04+5.0793509e-03j 1.1347564e-04+6.9278170e-04j]]


## 2) Applying a 2D FT (Fourier domain)

In [4]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 223.45734 +32.42344j    134.03648 +19.410824j    44.69067  +6.4799023j
   134.04053 +19.401508j ]
 [ 134.04454 +19.407856j    44.67924  +6.4852786j  -44.685078 -6.472818j
    44.65949  +6.481799j ]
 [  44.688095 +6.4725494j  -44.67673  -6.4731994j -134.03073 -19.425415j
   -44.678833 -6.4760504j]
 [ 134.03044 +19.428566j    44.663803 +6.462475j   -44.677658 -6.4670467j
    44.681606 +6.4781227j]]


#### Sanity check

In [5]:
img = np.fft.ifft2(visibilities).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[4.46827011e+01+6.4761119e+00j 4.46827011e+01+6.4761124e+00j
  7.00187683e-03+4.7672093e-03j 4.46827011e+01+6.4761114e+00j]
 [4.46827011e+01+6.4761119e+00j 1.17242336e-03+2.6136339e-03j
  6.93929195e-03+6.7490041e-03j 6.67494535e-03+5.7727396e-03j]
 [8.15200806e-03+5.8284402e-04j 1.46502256e-03+3.8327277e-03j
  3.48854065e-03+6.1569512e-03j 4.50688601e-03+4.3191016e-03j]
 [4.46827011e+01+6.4761119e+00j 3.99953127e-03+2.3147166e-03j
  3.19123268e-04+5.0791204e-03j 1.13129616e-04+6.9198012e-04j]]

Reals MSE: 1.13270314e-13
Imaginaries MSE: 1.3146031e-13


## 3) Data encoding: amplitude embedding (non-conventional domain)

In [6]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
n = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(n))) # set the number of qbits (no padding needed if outputs an integer=integer.0)
amp_dev = qml.device('default.qubit', wires)

@qml.qnode(amp_dev)
def amp_encoding(data):
    qml.AmplitudeEmbedding(data, wires) # normalize = True
    return qml.state() #qml.expval(qml.PauliZ(wires=wires))

results = amp_encoding(normalized_data).astype('complex64')
print(results)

print()

results = results*qml.math.sqrt(norm) # denormalization of the measurements outcomes
results = np.array(results).reshape(sky_image.shape[0] , sky_image.shape[1])
print(results)

[ 0.55334496+0.08028981j  0.33191308+0.04806681j  0.11066702+0.01604611j
  0.3319231 +0.04804375j  0.33193305+0.04805946j  0.11063872+0.01605942j
 -0.11065318-0.01602857j  0.11058981+0.01605081j  0.11066064+0.0160279j
 -0.11063251-0.01602951j -0.33189884-0.04810295j -0.11063771-0.01603657j
  0.33189812+0.04811075j  0.11060049+0.01600296j -0.1106348 -0.01601428j
  0.11064458+0.01604171j]

[[ 223.45732 +32.42344j    134.03648 +19.410824j    44.690666 +6.4799023j
   134.04053 +19.401508j ]
 [ 134.04454 +19.407856j    44.67924  +6.485278j   -44.685078 -6.4728174j
    44.65949  +6.4817986j]
 [  44.68809  +6.472549j   -44.67673  -6.4731994j -134.03073 -19.425415j
   -44.678833 -6.4760504j]
 [ 134.03044 +19.428564j    44.663803 +6.462475j   -44.677658 -6.4670467j
    44.681606 +6.4781227j]]


## 4) Applying 2D IFT (fidelity test)

In [7]:
img = np.fft.ifft2(results).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[4.46826973e+01+6.4761119e+00j 4.46827011e+01+6.4761119e+00j
  7.00044632e-03+4.7671497e-03j 4.46827011e+01+6.4761109e+00j]
 [4.46827011e+01+6.4761119e+00j 1.17185712e-03+2.6136637e-03j
  6.93812966e-03+6.7490339e-03j 6.67437911e-03+5.7727695e-03j]
 [8.15057755e-03+5.8296323e-04j 1.46406889e-03+3.8328469e-03j
  3.48711014e-03+6.1569512e-03j 4.50593233e-03+4.3192208e-03j]
 [4.46827011e+01+6.4761119e+00j 3.99914384e-03+2.3147464e-03j
  3.18378210e-04+5.0791502e-03j 1.12742186e-04+6.9200993e-04j]]

Reals MSE: 2.068067e-12
Imaginaries MSE: 1.5734954e-13


## Utils

### Quantum

### Classical